# Product Analysis Notebook: for MES conversion effort estimation

This notebook demonstrates how to perform a multi-step product analysis for a certain planning area to prepare for MES roll-out project.

### Step 1: Setup all supporting modules

First, we import the necessary libraries and use our helper function to get a direct `pyodbc` connection to the database.

In [215]:
import pandas as pd
import time
from database_connections import get_db_connection
import warnings

# --- TIMER START ---
start_time = time.time()
print("Analysis started. Timer initiated...")
# -------------------

# Add this line to ignore the specific UserWarning from pandas
warnings.filterwarnings(
    'ignore',
    category=UserWarning,
    module='pandas'
)

Analysis started. Timer initiated...


### Step 2: Run Initial Query and Create item list table `T_A00`

We define and execute our main SQL query for applicable item based on set of filter requirements. The results are loaded into a pandas DataFrame called `T_A00`. This DataFrame is now our temporary table, stored locally in the notebook's memory.

In [216]:
# Get a connection to the 'v12live' DSN
connection = get_db_connection('v12live')

T_A00 = pd.DataFrame()

sql_query = """
SELECT 
    T2.MBWHLO,
    T1.MMSTAT,
    T1.MMCHCD,
    T1.MMINDI,
    T2.MBSTAT,
    T2.MBRESP,
    T2.MBPUIT,
    T1.MMACRF,
    T1.MMITNO AS POPRNO,
    T1.MMITDS,
    T1.MMFUDS,
    T1.MMCFI3 AS PLC
FROM 
    MVXCDTA.MITMAS AS T1 
INNER JOIN 
    MVXCDTA.MITBAL AS T2 ON T1.MMITNO = T2.MBITNO AND T1.MMCONO = T2.MBCONO
WHERE 
    T2.MBWHLO = 'MF1' 
    AND T1.MMSTAT < '80' 
    AND T2.MBSTAT < '80' 
    AND T1.MMACRF <> 'CUSTREP'
    AND T2.MBRESP IN (
        'MP-4210', 'MP-4220', 'MP-4230', 'MP-4250', 'MP-4260', 
        'MP-4270', 'MP-4310', 'MP-4320', 'MP-4330', 'MP-4540'
    )
    AND T2.MBPUIT = 1
"""

if connection:
    try:
        print("--- Fetching initial data from database ---")
        T_A00 = pd.read_sql_query(sql_query, connection)
        
        # FIX: Standardize all column names to lowercase for consistency
        T_A00.columns = T_A00.columns.str.lower()
        
        print(f"✅ Query successful! Found {len(T_A00)} rows.")
        print("Data loaded into 'T_A00' DataFrame.")
        display(T_A00.head())
    except Exception as e:
        print(f"❌ Error executing query: {e}")
    finally:
        connection.close()
        print("\n🔌 Connection closed.")
else:
    print("⚠️ Cannot run query, no active database connection.")
    
print(f"The number of rows is: {len(T_A00)}")

✅ Connection successful to 'v12live' (v12Live).
--- Fetching initial data from database ---


C:\Users\tdnguyen\AppData\Local\Temp\ipykernel_8956\591859167.py:39: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  T_A00 = pd.read_sql_query(sql_query, connection)


✅ Query successful! Found 833 rows.
Data loaded into 'T_A00' DataFrame.


,mbwhlo,mmstat,mmchcd,mmindi,mbstat,mbresp,mbpuit,mmacrf,poprno,mmitds,mmfuds,plc
0,MF1,20,0.0,3.0,20,MP-4320,1.0,.6S,253994,KCD2-UT2-1,Signal converter KCD2-UT2-1 ...,300
1,MF1,20,0.0,3.0,20,MP-4220,1.0,.4S,814139,PCBA C 5500 DC Pwr Brd Int Mt,Assy Platine 103-1058.. Pwr B...,310
2,MF1,40,0.0,3.0,20,MP-4220,1.0,.4S,471662,PCBA C S890-DI890 Mother BD,BASE MODEL MOTHER BOARD DI890 8Ch ...,490
3,MF1,20,0.0,3.0,20,MP-4320,1.0,.6S,096045,KFA6-ER-1.5,Electrode relay KFA6-ER-1.5 ...,310
4,MF1,20,0.0,3.0,20,MP-4320,1.0,.6S,096046,KFA6-ER-1.6,Electrode relay KFA6-ER-1.6 ...,310



🔌 Connection closed.
The number of rows is: 833


### Step 3: Product structure check: single line, master or variant items in table `A01_Variant_Check`

This step groups item list based on planning area and product type in column `MMCHCD` to identify variant generator items

In [217]:
A01_Variant_Check = pd.DataFrame()
product_map = {
    0: "Single line items",
    2: 'Master items',
    3: "Variant items"
}
if not T_A00.empty:
    print("--- Creating A01_VariA01_Variant_Checkanten_Check ---")
    
    # Using lowercase column names now
    A01_Variant_Check = T_A00.groupby(['mbresp','mmchcd'])['poprno'].count().reset_index()
    A01_Variant_Check = A01_Variant_Check.rename(columns={'poprno': 'ItemCount'})
    A01_Variant_Check = A01_Variant_Check.rename(columns={'mmchcd': 'ProductType', 'mbresp': 'PlanningArea'})
    A01_Variant_Check['ProductType'] = A01_Variant_Check['ProductType'].replace(product_map)
    A01_Variant_Check = A01_Variant_Check.sort_values(by=['PlanningArea', 'ProductType'], ascending=True)
    
    print("✅ Analysis complete.")
    display(A01_Variant_Check.head(10))
else:
    print("⚠️ T_A00 DataFrame is empty, skipping analysis.")
    
print(f"The number of rows is: {len(A01_Variant_Check)}")

--- Creating A01_VariA01_Variant_Checkanten_Check ---
✅ Analysis complete.


,PlanningArea,ProductType,ItemCount
1,MP-4210,Master items,1
0,MP-4210,Single line items,237
2,MP-4210,Variant items,16
3,MP-4220,Single line items,207
5,MP-4250,Master items,1
4,MP-4250,Single line items,20
6,MP-4260,Single line items,50
7,MP-4270,Master items,2
8,MP-4270,Variant items,24
9,MP-4310,Single line items,41


The number of rows is: 13


### Step 4: Account control object check: in table `A02_ACO_Check`

This cell performs another aggregation on our initial DataFrame to count items based on planner code and the Acount Control Object `MMACRF` field to identify Finish Good (FG, .6x) and Semi-Finish Good (SFG, .4x)

In [218]:
A02_ACO_Check = pd.DataFrame()

if not T_A00.empty:
    print("--- Creating A02_ACO_Check ---")
    
    # Perform the groupby and count operation using lowercase names
    A02_ACO_Check = T_A00.groupby(['mbresp', 'mmacrf'])['poprno'].count().reset_index()
    
    # Rename the columns to match the SQL 'AS' clauses
    A02_ACO_Check = A02_ACO_Check.rename(columns={
        'mbresp': 'PlannerCode',
        'mmacrf': 'ACO',
        'poprno': 'ItemCount'
    })
    
    # Sort the results by MMACRF
    A02_ACO_Check = A02_ACO_Check.sort_values(by=['PlannerCode','ACO'])
    
    print("✅ Analysis complete.")
    display(A02_ACO_Check)
else:
    print("⚠️ T_A00 DataFrame is empty, skipping analysis.")
    
print(f"The number of rows is: {len(A02_ACO_Check)}")

--- Creating A02_ACO_Check ---
✅ Analysis complete.


,PlannerCode,ACO,ItemCount
0,MP-4210,.4S,100
1,MP-4210,.6S,154
2,MP-4220,.4S,114
3,MP-4220,.6S,93
4,MP-4250,.4S,18
5,MP-4250,.6M,1
6,MP-4250,.6S,2
7,MP-4260,.4S,1
8,MP-4260,.6S,49
9,MP-4270,.6S,26


The number of rows is: 19


### Step 5: Check item with lot control in table `A03_LotControled`

This step performs a more detailed aggregation, grouping by three columns to count items based on their lot control status (`MMINDI`). Any item with `MMINDI` = 0 is not yet lot controlled.

In [219]:
A03_LotControled = pd.DataFrame()

if not T_A00.empty:
    print("--- Creating A03_LotControled ---")
    
    # Perform the groupby operation on three columns using lowercase names
    A03_LotControled = T_A00.groupby(['mbresp', 'mmacrf', 'mmindi'])['poprno'].count().reset_index()
    
    # Rename the columns
    A03_LotControled = A03_LotControled.rename(columns={
        'mbresp': 'PlannerCode',
        'mmacrf': 'ACO',
        'mmindi': 'LotCtrolMethod',
        'poprno': 'LotCtrolCount'
    })
    
    # Sort the results
    A03_LotControled = A03_LotControled.sort_values(by=['PlannerCode', 'ACO', 'LotCtrolMethod'])
    
    print("✅ Analysis complete.")
    display(A03_LotControled)
else:
    print("⚠️ T_A00 DataFrame is empty, skipping analysis.")
    
print(f"The number of rows is: {len(A03_LotControled)}")

--- Creating A03_LotControled ---
✅ Analysis complete.


,PlannerCode,ACO,LotCtrolMethod,LotCtrolCount
0,MP-4210,.4S,0.0,5
1,MP-4210,.4S,1.0,14
2,MP-4210,.4S,3.0,81
3,MP-4210,.6S,0.0,1
4,MP-4210,.6S,3.0,153
5,MP-4220,.4S,0.0,21
6,MP-4220,.4S,1.0,1
7,MP-4220,.4S,3.0,92
8,MP-4220,.6S,3.0,93
9,MP-4250,.4S,3.0,18


The number of rows is: 26


### Step 6: Check product life cycle in `A03_PLCCheck`

This aggregation counts the number of items for each combination of planner and product line code (`PLC`). Item with `PLC` = [311,411] are subjected to transfer, while items with PLC >=490 are phased out.

In [220]:
A03_PLCCheck = pd.DataFrame()

if not T_A00.empty:
    print("--- Creating A03_PLCCheck ---")
    
    # Perform the groupby operation using lowercase names
    A03_PLCCheck = T_A00.groupby(['mbresp', 'plc'])['poprno'].count().reset_index()
    
    # Rename the columns
    A03_PLCCheck = A03_PLCCheck.rename(columns={
        'mbresp': 'PlannerCode',
        'plc': 'ProductLifeCycle',
        'poprno': 'ItemCount'
    })
    
    # Sort the results
    A03_PLCCheck = A03_PLCCheck.sort_values(by=['PlannerCode','ProductLifeCycle'])
    
    print("✅ Analysis complete.")
    display(A03_PLCCheck)
else:
    print("⚠️ T_A00 DataFrame is empty, skipping analysis.")
    
print(f"The number of rows is: {len(A03_PLCCheck)}")

--- Creating A03_PLCCheck ---
✅ Analysis complete.


,PlannerCode,ProductLifeCycle,ItemCount
0,MP-4210,200,8
1,MP-4210,240,9
2,MP-4210,241,1
3,MP-4210,310,217
4,MP-4210,410,3
5,MP-4210,490,6
6,MP-4210,510,8
7,MP-4210,520,1
8,MP-4210,INT,1
9,MP-4220,200,5


The number of rows is: 49


### Step 7: Generate full product routing by linking item table `T_A0O` to database & create routing table `T_A10`

This step links our initial DataFrame (`T_A00`) with two new database tables (`MPDOPE` and `MPDHED`).

1.  We extract the unique item numbers (`POPRNO`) from our DataFrame.
2.  We use these item numbers to build a new SQL query that efficiently fetches only the required data from the database.
3.  We execute this query and get a new DataFrame with the details.
4.  Finally, we perform a `merge` (join) in pandas to combine our original data with the new details, creating the final `T_A10` DataFrame.

In [221]:
T_A10 = pd.DataFrame() # Initialize T_A10 as an empty DataFrame

if not T_A00.empty:
    print("--- Linking DataFrame to MPDOPE and MPDHED tables ---")
    
    # 1. Extract the unique keys from your DataFrame using the lowercase name
    item_numbers = T_A00['poprno'].unique().tolist()
    
    if item_numbers:
        # 2. Format the list of keys for the SQL 'IN' clause
        formatted_keys = ", ".join([f"'{item}'" for item in item_numbers])

        # 3. Construct the new SQL query
        details_sql_query = f"""
        SELECT 
            T_OPE.POPRNO, T_HED.PHSTAT, T_OPE.POSTRT, T_OPE.POOPNO, T_OPE.POPLGR, 
            T_OPE.POOPDS, T_OPE.POTXT1, T_OPE.POTXT2, T_OPE.PODOID, T_OPE.POAURP, 
            T_OPE.POCONO, T_OPE.POFACI
        FROM 
            MVXCDTA.MPDOPE AS T_OPE
        INNER JOIN 
            MVXCDTA.MPDHED AS T_HED ON T_OPE.POPRNO = T_HED.PHPRNO 
                                    AND T_OPE.POFACI = T_HED.PHFACI 
                                    AND T_OPE.POCONO = T_HED.PHCONO
        WHERE 
            T_OPE.POSTRT = 'STD'
            AND T_OPE.POCONO = 1
            AND T_OPE.POFACI = 'MF1'
            AND T_OPE.POTDAT = 99999999
            AND T_OPE.POPRNO IN ({formatted_keys})
        """

        # 4. Execute the query to get details
        df_details = pd.DataFrame()
        connection = get_db_connection('v12live') # Re-open a connection
        if connection:
            try:
                print(f"Fetching details for {len(item_numbers)} items...")
                df_details = pd.read_sql_query(details_sql_query, connection)
                
                # Standardize details columns to lowercase
                df_details.columns = df_details.columns.str.lower()
                
                print(f"✅ Query successful! Found {len(df_details)} matching detail rows.")

                # 5. Perform the final join in pandas to create T_A10
                T_A10 = pd.merge(
                    left=T_A00,
                    right=df_details,
                    on='poprno', # Join on the lowercase common item number column
                    how='inner'  # Use 'inner' to match the SQL INNER JOIN
                )
                
                # Select and reorder columns, including 'mbresp' for the next step
                T_A10 = T_A10[[
                    'plc', 'poprno', 'mmacrf', 'mbpuit', 'phstat', 'postrt', 
                    'poopno', 'poplgr', 'poopds', 'potxt1', 'potxt2', 'podoid', 'poaurp', 
                    'pocono', 'pofaci', 'mbresp'
                ]]

                print("\n--- Final T_A10 DataFrame created ---")
                display(T_A10.head())

            except Exception as e:
                print(f"❌ Error executing the details query: {e}")
            finally:
                connection.close()
                print("\n🔌 Connection closed.")
    else:
        print("⚠️ No item numbers in T_A00 to use for the next query.")
else:
    print("⚠️ T_A00 DataFrame is empty, skipping the linking step.")
    

--- Linking DataFrame to MPDOPE and MPDHED tables ---


✅ Connection successful to 'v12live' (v12Live).
Fetching details for 833 items...


C:\Users\tdnguyen\AppData\Local\Temp\ipykernel_8956\2830515060.py:39: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_details = pd.read_sql_query(details_sql_query, connection)


✅ Query successful! Found 27991 matching detail rows.

--- Final T_A10 DataFrame created ---


,plc,poprno,mmacrf,mbpuit,phstat,postrt,poopno,poplgr,poopds,potxt1,potxt2,podoid,poaurp,pocono,pofaci,mbresp
0,300,253994,.6S,1.0,20,STD,10.0,4320 M,MATERIAL KITTING,...,Product transfer from Interface back to Analog...,,1.0,1.0,MF1,MP-4320
1,300,253994,.6S,1.0,20,STD,1000.0,4313 K,KANBAN KITTING INTERFACE PRINT,...,...,,1.0,1.0,MF1,MP-4320
2,300,253994,.6S,1.0,20,STD,1100.0,4324,PRINTING TAMPON 1,PlanTimeYr- PRT-A261 and PRT-A264 (SMT) ...,...,260-5767E,2.0,1.0,MF1,MP-4320
3,300,253994,.6S,1.0,20,STD,1110.0,4320,AID/RESOURCE/FIXTURE,...,...,65-2767A,1.0,1.0,MF1,MP-4320
4,300,253994,.6S,1.0,20,STD,1300.0,4117 K,KANBAN KITTING MATL FORMING,...,...,,1.0,1.0,MF1,MP-4320



🔌 Connection closed.


### Step 8: Consolidate total Document used inside routing in table `T_A11`

This step creates the `T_A11` table by aggregating the `T_A10` data. It trims whitespace from the document ID, groups by planner and the cleaned ID, and counts the number of document ID.

In [222]:
T_A11 = pd.DataFrame() # Initialize T_A11 as an empty DataFrame

if not T_A10.empty:
    print("--- Creating T_A11 by grouping T_A10 ---")
    
    # Create a working copy to avoid SettingWithCopyWarning
    temp_df = T_A10.copy()
    
    # Trim whitespace from the 'podoid' column, handling potential non-string data
    temp_df['m3doid'] = temp_df['podoid'].astype(str).str.strip()
    
    # Perform the groupby operation
    T_A11 = temp_df.groupby(['mbresp', 'm3doid'])['poopno'].count().reset_index()
    
    # Rename the columns to match the desired output
    T_A11 = T_A11.rename(columns={
        'mbresp': 'PlannerCode',
        'm3doid': 'M3DOID',
        'poopno': 'UsageCnt'
    })
    
    print("✅ T_A11 DataFrame created successfully.")
    display(T_A11.head())
else:
    print("⚠️ T_A10 DataFrame is empty, skipping creation of T_A11.")
    
print(f"The number of rows is: {len(T_A11)}")

--- Creating T_A11 by grouping T_A10 ---
✅ T_A11 DataFrame created successfully.


,PlannerCode,M3DOID,UsageCnt
0,MP-4210,,819
1,MP-4210,01-6463B,1
2,MP-4210,01-6760B,1
3,MP-4210,01-6785D,4
4,MP-4210,01-6800D,2


The number of rows is: 5600


### Step 9: Connect M3DOID to EDM database `EDMEWAREAD` to create `A12_DoKID_EDM`
This step connects to a different database (EDMEWAREAD) to enrich our T_A11 data with basic information `Info1` and `Info2`

1. We extract the unique document IDs (M3DOID) from T_A11.
2. We connect to the edw DSN.
3. We query the ADMEDP_EDM_DOCS table using these IDs, breaking them into chunks to avoid database errors (each trunk = 999 items because limit = 1000 items).
4. We perform a final merge (join) in pandas to combine the usage counts with the document details.

In [223]:
A12_DoKID_EDM = pd.DataFrame() # Initialize as empty
if not T_A11.empty:
    print("--- Joining T_A11 with ADMEDP_EDM_DOCS from EDW database ---")
    
    # 1. Extract the unique document IDs from T_A11 and filter out empty strings
    doc_ids = [doc_id for doc_id in T_A11['M3DOID'].unique().tolist() if doc_id]
    
    if doc_ids:
        # 2. Connect to the 'edw' database
        edw_connection = get_db_connection('edw')
        
        if edw_connection:
            df_docs_details_list = []
            # FIX: Break the list of IDs into chunks of 999 to avoid Oracle DB error
            chunk_size = 999
            id_chunks = [doc_ids[i:i + chunk_size] for i in range(0, len(doc_ids), chunk_size)]
            
            try:
                for i, chunk in enumerate(id_chunks):
                    print(f"Fetching document details from EDW: chunk {i+1}/{len(id_chunks)}")
                    formatted_doc_ids = ", ".join([f"'{doc_id}'" for doc_id in chunk])
                    
                    # 3. Construct the query for the EDW database
                    docs_sql_query = f"""
                    SELECT 
                        DOCNUMBER, 
                        INFO, 
                        INFO1
                    FROM 
                        ADMEDP.EDM_DOCS
                    WHERE 
                        DOCNUMBER IN ({formatted_doc_ids})
                    """
                    df_chunk = pd.read_sql_query(docs_sql_query, edw_connection)
                    df_docs_details_list.append(df_chunk)
                
                # Combine all chunks into one DataFrame
                df_docs_details = pd.concat(df_docs_details_list, ignore_index=True)
                df_docs_details.columns = df_docs_details.columns.str.lower()
                print(f"✅ Query successful! Found {len(df_docs_details)} matching documents in total.")
                
                # 4. Join the T_A11 data with the document details
                A12_DoKID_EDM = pd.merge(
                    left=T_A11,
                    right=df_docs_details,
                    left_on='M3DOID',
                    right_on='docnumber',
                    how='inner'
                )
                
                # Sort the final results
                A12_DoKID_EDM = A12_DoKID_EDM.sort_values(by='M3DOID')
                
                print("\n--- Final Enriched Data --- ")
                display(A12_DoKID_EDM.head())

            except Exception as e:
                print(f"❌ Error executing EDW query: {e}")
            finally:
                edw_connection.close()
                print("\n🔌 EDW Connection closed.")
else:
    print("⚠️ T_A11 DataFrame is empty, skipping final join.")
    
print(f"The number of rows is: {len(A12_DoKID_EDM)}")

--- Joining T_A11 with ADMEDP_EDM_DOCS from EDW database ---
✅ Connection successful to 'edw' (EDMEWAREAD).
Fetching document details from EDW: chunk 1/6


C:\Users\tdnguyen\AppData\Local\Temp\ipykernel_8956\3371025336.py:34: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_chunk = pd.read_sql_query(docs_sql_query, edw_connection)


Fetching document details from EDW: chunk 2/6
Fetching document details from EDW: chunk 3/6
Fetching document details from EDW: chunk 4/6
Fetching document details from EDW: chunk 5/6
Fetching document details from EDW: chunk 6/6
✅ Query successful! Found 5315 matching documents in total.

--- Final Enriched Data --- 


,PlannerCode,M3DOID,UsageCnt,docnumber,info,info1
3869,MP-4320,01-5616,2,01-5616,schematic KFA.-ER-1.. \r\n(Elektrodenrelais / ...,11.10.01 A. Hartnagel PA-EW 50-2627
4597,MP-4540,01-6366,15,01-6366,schematic to ISTA-2**-BP** single baseplate,02.09.02 PA-EW A. Hartnagel
4598,MP-4540,01-6367,7,01-6367,schematic to ISTA-2**-BP**-R redundant baseplate,02.09.02 PA-EW A. Hartnagel
4599,MP-4540,01-6368A,1,01-6368A,schematic to PCBA T ISTA-BP MATRIX 1S (for FBM...,None
4600,MP-4540,01-6369A,1,01-6369A,schematic to PCBA T ISTA-BP MATRIX 2S (for FBM...,None



🔌 EDW Connection closed.
The number of rows is: 5591


### Step 10: Get all files inside each document ID in table `A13_Doc_type`

This step enriches the data further by connecting to the `EDMEWAREAD` database again to get file-specific details from the `ADMEDP_EDM_FILES` table.

1.  We extract the unique document IDs (`M3DOID`) from `A12_DoKID_EDM`.
2.  We query the `ADMEDP_EDM_FILES` table for matching documents.
3.  We join the results and then create two new columns: `FileExt` by extracting the file extension from the filename

In [224]:
A13_Doc_type = pd.DataFrame() # Initialize as empty
if not A12_DoKID_EDM.empty:
    print("--- Joining with ADMEDP_EDM_FILES from EDW database ---")
    
    # 1. Extract the unique document IDs from A12_DoKID_EDM
    doc_ids = A12_DoKID_EDM['M3DOID'].unique().tolist()
    
    if doc_ids:
        # 2. Connect to the 'edw' database
        edw_connection = get_db_connection('edw')
        
        if edw_connection:
            df_files_details_list = []
            #Break the list of IDs into chunks of 999
            chunk_size = 999
            id_chunks = [doc_ids[i:i + chunk_size] for i in range(0, len(doc_ids), chunk_size)]
            
            try:
                for i, chunk in enumerate(id_chunks):
                    print(f"Fetching file details from EDW: chunk {i+1}/{len(id_chunks)}")
                    formatted_doc_ids = ", ".join([f"'{doc_id}'" for doc_id in chunk])
                    
                    # Construct the query for the EDW database
                    files_sql_query = f"""
                    SELECT 
                        DOCNUMBER, 
                        FILENAME, 
                        FILEUSER
                    FROM 
                        ADMEDP.EDM_FILES
                    WHERE 
                        DOCNUMBER IN ({formatted_doc_ids})
                    """
                    df_chunk = pd.read_sql_query(files_sql_query, edw_connection)
                    df_files_details_list.append(df_chunk)

                # Combine all chunks into one DataFrame
                df_files_details = pd.concat(df_files_details_list, ignore_index=True)
                df_files_details.columns = df_files_details.columns.str.lower()
                print(f"✅ Query successful! Found {len(df_files_details)} matching files.")
                
                # Join the A12 data with the file details
                merged_df = pd.merge(
                    left=A12_DoKID_EDM,
                    right=df_files_details,
                    left_on='M3DOID',
                    right_on='docnumber',
                    how='inner'
                )
                
                # 3. Create the new columns
                # Replicate: Mid([FILENAME],InStr(1,Trim([FILENAME]),".")+1,3)
                merged_df['FileExt'] = merged_df['filename'].str.strip().str.split('.').str[-1].str.slice(0, 3)            
                
                # Select and reorder final columns
                A13_Doc_type = merged_df[[
                    'PlannerCode', 'M3DOID', 'UsageCnt', 'info', 'info1', 
                    'FileExt', 'fileuser'
                ]]
                
                print("\n--- Final A13_Doc_type DataFrame Created ---")
                display(A13_Doc_type.head())

            except Exception as e:
                print(f"❌ Error executing EDW query: {e}")
            finally:
                edw_connection.close()
                print("\n🔌 EDW Connection closed.")
else:
    print("⚠️ A12_DoKID_EDM DataFrame is empty, skipping final join.")
    
print(f"The number of rows is: {len(A13_Doc_type)}")

--- Joining with ADMEDP_EDM_FILES from EDW database ---
✅ Connection successful to 'edw' (EDMEWAREAD).
Fetching file details from EDW: chunk 1/6


C:\Users\tdnguyen\AppData\Local\Temp\ipykernel_8956\625172657.py:34: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_chunk = pd.read_sql_query(files_sql_query, edw_connection)


Fetching file details from EDW: chunk 2/6
Fetching file details from EDW: chunk 3/6
Fetching file details from EDW: chunk 4/6
Fetching file details from EDW: chunk 5/6
Fetching file details from EDW: chunk 6/6
✅ Query successful! Found 12353 matching files.

--- Final A13_Doc_type DataFrame Created ---


,PlannerCode,M3DOID,UsageCnt,info,info1,FileExt,fileuser
0,MP-4320,01-5616,2,schematic KFA.-ER-1.. \r\n(Elektrodenrelais / ...,11.10.01 A. Hartnagel PA-EW 50-2627,pdf,282.0
1,MP-4320,01-5616,2,schematic KFA.-ER-1.. \r\n(Elektrodenrelais / ...,11.10.01 A. Hartnagel PA-EW 50-2627,sch,282.0
2,MP-4320,01-5616,2,schematic KFA.-ER-1.. \r\n(Elektrodenrelais / ...,11.10.01 A. Hartnagel PA-EW 50-2627,txt,282.0
3,MP-4540,01-6366,15,schematic to ISTA-2**-BP** single baseplate,02.09.02 PA-EW A. Hartnagel,txt,282.0
4,MP-4540,01-6366,15,schematic to ISTA-2**-BP** single baseplate,02.09.02 PA-EW A. Hartnagel,sch,282.0



🔌 EDW Connection closed.
The number of rows is: 13084


### Step 11: Pivot Data to Analyze File Types in table `A14_Doc_type`
This step replicates a crosstab query (similar to a PIVOT in SQL) to transform the `A13_Doc_type` DataFrame. We will use the powerful pivot_table function from pandas to achieve this.

The operation reorganizes the data so that each unique file extension (FileExt) becomes its own column. The values in these new columns will represent the count of each file type for a given document, which is perfect for quickly analyzing the composition of files within each document ID.

In [225]:
A14_Doc_type = pd.DataFrame() # Initialize the final DataFrame

if not A13_Doc_type.empty:
    print("--- Grouping by M3DOID and counting file types ---")
    
    try:
        # Step 1: Create the indicator columns for each file type (like the previous step)
        df_with_dummies = pd.get_dummies(A13_Doc_type, columns=['FileExt'], prefix='', prefix_sep='')

        # Get the names of the newly created file extension columns
        original_cols = A13_Doc_type.columns.tolist()
        dummy_cols = [col for col in df_with_dummies.columns if col not in original_cols]

        # Step 2: Define how to aggregate each column
        # We'll take the first value for info columns (assuming they're the same for each M3DOID)
        # and we'll sum the new dummy columns to get a count.
        agg_operations = {
            'PlannerCode': 'first',
            'UsageCnt': 'first',
            'info': 'first',
            'info1': 'first',
            'fileuser': 'first',
        }
        for col in dummy_cols:
            agg_operations[col] = 'sum'
            
        # Step 3: Group by M3DOID and apply the aggregations
        A14_Doc_type = df_with_dummies.groupby('M3DOID').agg(agg_operations).reset_index()

        print("✅ Aggregation successful. A14_Doc_type created.")
        display(A14_Doc_type.head())

    except Exception as e:
        print(f"❌ Error during aggregation: {e}")
else:
    print("⚠️ A13_Doc_type is empty, skipping the aggregation.")

print(f"The number of rows is: {len(A14_Doc_type)}")

# filtered_df = A14_Doc_type[A14_Doc_type['M3DOID'] == '10-AG93A']
# filtered_df = filtered_df[['M3DOID', 'PlannerCode', 'UsageCnt', 'info', 'info1', 'cab']]
# filtered_df

--- Grouping by M3DOID and counting file types ---


✅ Aggregation successful. A14_Doc_type created.


,M3DOID,PlannerCode,UsageCnt,info,info1,fileuser,1,2,3,7z,...,tst,txt,vlm,vsd,xls,xml,xuv,zip,zpl,zpo
0,01-5616,MP-4320,2,schematic KFA.-ER-1.. \r\n(Elektrodenrelais / ...,11.10.01 A. Hartnagel PA-EW 50-2627,282.0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,01-6366,MP-4540,15,schematic to ISTA-2**-BP** single baseplate,02.09.02 PA-EW A. Hartnagel,282.0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,01-6367,MP-4540,7,schematic to ISTA-2**-BP**-R redundant baseplate,02.09.02 PA-EW A. Hartnagel,282.0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,01-6368A,MP-4540,1,schematic to PCBA T ISTA-BP MATRIX 1S (for FBM...,None,1941.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,01-6369A,MP-4540,1,schematic to PCBA T ISTA-BP MATRIX 2S (for FBM...,None,1941.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


The number of rows is: 5312


### Step 12: Create a Clean Copy as `A15_EDM_Doc_ext_type`
The provided SQL query simply selects all columns and rows from the `A14_Doc-type` table. The equivalent action in pandas is to create a copy of the `A14_Doc_type` DataFrame.

We'll name this new DataFrame `A15_EDM_Doc_ext_type` to save the results from the previous aggregation and prepare for the next stage of the analysis. It's important to use the .copy() method to ensure the new DataFrame is independent of the original.

In [226]:
A15_EDM_Doc_ext_type = pd.DataFrame() # Initialize the DataFrame

if not A14_Doc_type.empty:
    print("--- Creating a copy of A14_Doc_type ---")
    
    # The SQL query is effectively a SELECT *, so we just copy the DataFrame.
    A15_EDM_Doc_ext_type = A14_Doc_type.copy()
    
    print("✅ DataFrame copied successfully to A15_EDM_Doc_ext_type.")
    display(A15_EDM_Doc_ext_type.head())
else:
    print("⚠️ A14_Doc_type is empty, so an empty A15_EDM_Doc_ext_type was created.")

print(f"The number of rows is: {len(A15_EDM_Doc_ext_type)}")

--- Creating a copy of A14_Doc_type ---
✅ DataFrame copied successfully to A15_EDM_Doc_ext_type.


,M3DOID,PlannerCode,UsageCnt,info,info1,fileuser,1,2,3,7z,...,tst,txt,vlm,vsd,xls,xml,xuv,zip,zpl,zpo
0,01-5616,MP-4320,2,schematic KFA.-ER-1.. \r\n(Elektrodenrelais / ...,11.10.01 A. Hartnagel PA-EW 50-2627,282.0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,01-6366,MP-4540,15,schematic to ISTA-2**-BP** single baseplate,02.09.02 PA-EW A. Hartnagel,282.0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,01-6367,MP-4540,7,schematic to ISTA-2**-BP**-R redundant baseplate,02.09.02 PA-EW A. Hartnagel,282.0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,01-6368A,MP-4540,1,schematic to PCBA T ISTA-BP MATRIX 1S (for FBM...,None,1941.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,01-6369A,MP-4540,1,schematic to PCBA T ISTA-BP MATRIX 2S (for FBM...,None,1941.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


The number of rows is: 5312


### Step 13: Join with User Data and Filter for CAB Files
This step enriches our data by joining it with a table of user information (`T_FileUser`). We'll then filter the results to isolate only the documents that contain `.cab` files (a common file type for labels or packages) and sort the final output by the user ID.

First, we need to create the `T_FileUser` DataFrame from the data you provided. Then, we'll perform the join and filter operations.

In [227]:
import pandas as pd

# Initialize the final DataFrame
A16_EDM_Doc_CAB_Labels = pd.DataFrame()

# --- Create the T_FileUser DataFrame from your data ---
file_user_data = {
    'fileuser': [152, 270, 964, 1337, 2539, 6091],
    'Name': [
        'Klauer, Sonja', 
        'Szekely, Beatrice', 
        'Kusche, Britta', 
        'Gosenheimer, Dieter', 
        'Reinig-Seidensticker, Carmen', 
        'Darm, Wenmarie'
    ]
}
T_FileUser = pd.DataFrame(file_user_data)
# ---

if not A15_EDM_Doc_ext_type.empty:
    print("--- Joining with T_FileUser and filtering for 'cab' files ---")
    
    # Check if the 'cab' column exists before trying to filter
    if 'cab' in A15_EDM_Doc_ext_type.columns:
        
        # Create a working copy
        temp_df = A15_EDM_Doc_ext_type.copy()
        
        # --- 1. LEFT JOIN ---
        # Ensure the join keys have the same data type.
        temp_df['fileuser'] = temp_df['fileuser'].astype('Int64') # Handles potential missing values
        T_FileUser['fileuser'] = T_FileUser['fileuser'].astype('Int64')
        
        # Perform the left merge, equivalent to a LEFT JOIN
        merged_df = pd.merge(
            left=temp_df,
            right=T_FileUser,
            on='fileuser',
            how='left'
        )
        
        # --- 2. WHERE clause ---
        # Filter the rows where the 'cab' column has a value of 1
        filtered_df = merged_df[merged_df['cab'] >= 1].copy()
        
        # --- 3. ORDER BY clause ---
        # Sort the results by the FILEUSER column
        sorted_df = filtered_df.sort_values(by='fileuser')
        
        # --- 4. SELECT clause ---
        # Define the final list of columns to keep.
        # Note: 'DOCNUMBER' was in your query but is not in the A15 DataFrame, so it's excluded.
        final_columns = [
            'PlannerCode', 'M3DOID', 'UsageCnt', 'INFO', 'INFO1', 'fileuser', 'Name',
            'cab', 'cas', 'cdr', 'eti', 'fmt', 'jpg', 'pdf', 'png', 'zip', 'zzz'
        ]
        
        # Filter the list to only include columns that actually exist in the DataFrame
        existing_columns = [col for col in final_columns if col in sorted_df.columns]
        
        A16_EDM_Doc_CAB_Labels = sorted_df[existing_columns]
        A16_EDM_Doc_CAB_Labels = A16_EDM_Doc_CAB_Labels.sort_values([ 'PlannerCode', 'M3DOID'])

        print("✅ Join and filter operation successful.")
        display(A16_EDM_Doc_CAB_Labels.head())

    else:
        print("⚠️ Column 'cab' not found in A15_EDM_Doc_ext_type. No filtering performed.")
        print("The resulting DataFrame will be empty.")

else:
    print("⚠️ A15_EDM_Doc_ext_type is empty, skipping operation.")

print(f"The number of rows is: {len(A16_EDM_Doc_CAB_Labels)}")

--- Joining with T_FileUser and filtering for 'cab' files ---
✅ Join and filter operation successful.


,PlannerCode,M3DOID,UsageCnt,fileuser,Name,cab,cas,cdr,eti,fmt,jpg,pdf,png,zip
1004,MP-4210,10-AKE3G,2,152,"Klauer, Sonja",1,0,1,0,1,0,1,1,0
1007,MP-4210,10-AM58F,2,152,"Klauer, Sonja",1,0,1,0,1,0,1,1,0
1008,MP-4210,10-AM61K,1,270,"Szekely, Beatrice",1,0,1,0,1,0,1,0,0
1021,MP-4210,10-ATW8F,7,270,"Szekely, Beatrice",1,0,1,0,1,1,1,0,0
1080,MP-4210,10-BFC2A,2,152,"Klauer, Sonja",1,0,1,0,1,0,1,1,0


The number of rows is: 212


### Step 14: Filter for '916' Documents and Generate URLs
This step filters the dataset to find all documents where the ID (`M3DOID`) starts with "916". Then, it constructs a unique URL for each document by combining a base address with the document's ID. Finally, it selects a specific set of columns and sorts the result.

In [228]:
# Initialize the final DataFrame
A17_916_Files = pd.DataFrame()

if not A15_EDM_Doc_ext_type.empty:
    print("--- Filtering for M3DOID starting with '916' and creating URLs ---")
    
    try:
        # Create a working copy
        temp_df = A15_EDM_Doc_ext_type.copy()

        # --- 1. WHERE clause ---
        # Filter rows where M3DOID starts with "916". We use na=False to handle potential non-string values.
        filtered_df = temp_df[temp_df['M3DOID'].astype(str).str.startswith('916', na=False)].copy()

        # --- 2. Create the new URL column ---
        if not filtered_df.empty:
            base_url = "https://pfde-docs-prd.eu.p-f.biz/service-edocs/DocumentRestService.svc/document/"
            
            # Replicate the string manipulation from the SQL query using .str methods
            # Here, we remove the hyphen from the M3DOID for the last part of the URL
            doc_id_no_hyphen = filtered_df['M3DOID'].str.replace('-', '', n=1)
            
            # Use f-string formatting to build the full URL
            filtered_df['URL'] = (
                f"{base_url}" + 
                filtered_df['M3DOID'].str.strip() + 
                "/txt/" + 
                doc_id_no_hyphen
            )

            # --- 3. ORDER BY clause ---
            # Sort the DataFrame by PlannerCode
            sorted_df = filtered_df.sort_values(by='PlannerCode')
            
            # --- 4. SELECT clause ---
            # Define the final list of columns to keep.
            final_columns = [
                'PlannerCode', 'M3DOID', 'UsageCnt', 'INFO', 
                'INFO1', 'FILEUSER', 'pcx', 'txt', 'URL'
            ]
            
            # Ensure we only select columns that actually exist in the DataFrame to prevent errors
            existing_columns = [col for col in final_columns if col in sorted_df.columns]
            
            A17_916_Files = sorted_df[existing_columns]
            A17_916_Files = A17_916_Files.sort_values([ 'PlannerCode', 'M3DOID'])
            
            print("✅ Filter, URL generation, and sort completed successfully.")
            display(A17_916_Files.head())
        else:
            print("ℹ️ No documents found with an M3DOID starting with '916'.")

    except Exception as e:
        print(f"❌ An error occurred: {e}")

else:
    print("⚠️ A15_EDM_Doc_ext_type is empty, skipping operation.")

print(f"The number of rows is: {len(A17_916_Files)}")

--- Filtering for M3DOID starting with '916' and creating URLs ---
✅ Filter, URL generation, and sort completed successfully.


,PlannerCode,M3DOID,UsageCnt,pcx,txt,URL
4209,MP-4210,916-0002B,6,3,1,https://pfde-docs-prd.eu.p-f.biz/service-edocs...
4210,MP-4210,916-0014,2,2,1,https://pfde-docs-prd.eu.p-f.biz/service-edocs...
4212,MP-4210,916-0099,2,2,1,https://pfde-docs-prd.eu.p-f.biz/service-edocs...
4213,MP-4210,916-0100D,2,3,1,https://pfde-docs-prd.eu.p-f.biz/service-edocs...
4217,MP-4210,916-0399,1,2,1,https://pfde-docs-prd.eu.p-f.biz/service-edocs...


The number of rows is: 209


### Step 15: Select and Sort Key Document Data
This step creates a new, focused DataFrame named A18_EDM_Doc_KeyValue_File. The goal is to select a specific subset of columns from the A15_EDM_Doc_ext_type table and then sort the results based on the M3DOID to ensure a consistent order. This is a common step for cleaning up a table and preparing it for final review or export.

In [229]:
# Initialize the final DataFrame
A18_EDM_Doc_KeyValue_File = pd.DataFrame()

if not A15_EDM_Doc_ext_type.empty:
    print("--- Selecting specific columns and sorting by M3DOID ---")
    
    try:
        # --- 1. SELECT clause ---
        # Define the list of columns you want to keep from the SQL query.
        # As noted, 'DOCNUMBER' is not in our DataFrame and will be ignored.
        final_columns = [
            'PlannerCode', 'M3DOID', 'UsageCnt', 'INFO', 'INFO1', 'FILEUSER',
            'bmp', 'cab', 'cas', 'doc', 'dwg', 'eti', 'fmt', 'jpg', 'pcx',
            'pdf', 'stp', 'txt', 'xls', 'zip', 'zzz'
        ]
        
        # We create a final list of columns that actually exist in the A15 DataFrame.
        # This prevents errors if some file extension columns (e.g., 'bmp') were never created.
        existing_columns = [col for col in final_columns if col in A15_EDM_Doc_ext_type.columns]
        
        # Create a new DataFrame with only the selected columns.
        selected_df = A15_EDM_Doc_ext_type[existing_columns]
        
        # --- 2. ORDER BY clause ---
        # Sort the new DataFrame by the M3DOID column.
        A18_EDM_Doc_KeyValue_File = selected_df.sort_values(by='M3DOID').reset_index(drop=True)
        A18_EDM_Doc_KeyValue_File = A18_EDM_Doc_KeyValue_File.sort_values([ 'PlannerCode', 'M3DOID'])
        
        print("✅ Column selection and sorting completed successfully.")
        display(A18_EDM_Doc_KeyValue_File.head())

    except Exception as e:
        print(f"❌ An error occurred: {e}")

else:
    print("⚠️ A15_EDM_Doc_ext_type is empty, skipping operation.")

print(f"The number of rows is: {len(A18_EDM_Doc_KeyValue_File)}")

--- Selecting specific columns and sorting by M3DOID ---
✅ Column selection and sorting completed successfully.


,PlannerCode,M3DOID,UsageCnt,bmp,cab,cas,doc,dwg,eti,fmt,jpg,pcx,pdf,stp,txt,xls,zip
8,MP-4210,01-6463B,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
12,MP-4210,01-6760B,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
13,MP-4210,01-6785D,4,0,0,0,0,0,0,0,0,0,1,0,0,0,0
14,MP-4210,01-6800D,2,0,0,0,0,0,0,0,0,0,1,0,0,0,0
15,MP-4210,01-6802A,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0


The number of rows is: 5312


### Step 16: Link Operations to Work Center Descriptions to make AprisoWorkCenter AWC
This step identifies specific manufacturing operations by filtering `T_A10` for rows where `POAURP` is 2. It then joins this data with the `MVXCDTA_MPDWCT` database table to pull in the work center description (`PPPLGD`). This allows us to see the human-readable name for the work centers associated with these specific operations.

In [230]:
# Initialize the final DataFrame
A20_AprisoWorkCenter = pd.DataFrame()

if not T_A10.empty:
    print("--- Joining T_A10 with MPDWCT to get Work Center details ---")
    
    # --- 1. WHERE clause: Filter T_A10 first for efficiency ---
    t10_filtered = T_A10[T_A10['poaurp'] == 2].copy()
    
    if not t10_filtered.empty:
        # --- 2. Get the necessary data from the database ---
        # Get the unique work center codes (poplgr) to query the database efficiently
        work_center_codes = t10_filtered['poplgr'].unique().tolist()
        formatted_codes = ", ".join([f"'{code}'" for code in work_center_codes])

        # Construct the SQL query
        db_query = f"""
        SELECT PPPLGR, PPPLGD, PPCONO, PPFACI
        FROM MVXCDTA.MPDWCT
        WHERE PPPLGR IN ({formatted_codes})
        """
        
        df_mpdwct = pd.DataFrame()
        connection = get_db_connection('v12live') # Assuming 'v12live' is the correct DSN
        if connection:
            try:
                print(f"Fetching details for {len(work_center_codes)} work centers from the database...")
                df_mpdwct = pd.read_sql_query(db_query, connection)
                
                # Standardize column names to lowercase for consistent merging
                df_mpdwct.columns = df_mpdwct.columns.str.lower()
                print(f"✅ Found {len(df_mpdwct)} matching work centers.")
                
            except Exception as e:
                print(f"❌ Error executing database query: {e}")
            finally:
                connection.close()
                print("🔌 Connection closed.")
        
        # --- 3. INNER JOIN ---
        # Proceed only if we successfully fetched data from the database
        if not df_mpdwct.empty:
            # Note: In your notebook, 'PlannerCode' corresponds to the 'mbresp' column.
            # We will select 'mbresp' from T_A10 and rename it later.
            
            # Merge the filtered T_A10 data with the work center details
            A20_AprisoWorkCenter = pd.merge(
                left=t10_filtered,
                right=df_mpdwct,
                left_on=['pocono', 'pofaci', 'poplgr'],
                right_on=['ppcono', 'ppfaci', 'ppplgr'],
                how='inner'
            )
            
            # --- 4. SELECT Clause ---
            # Select and rename the final columns to match the query output
            A20_AprisoWorkCenter = A20_AprisoWorkCenter.rename(columns={'mbresp': 'PlannerCode'})
            final_columns = ['PlannerCode', 'ppplgd', 'poopds', 'poaurp']
            A20_AprisoWorkCenter = A20_AprisoWorkCenter[final_columns]
            
            print("\n✅ Join successful. Final DataFrame created.")
            display(A20_AprisoWorkCenter.head())
        else:
            print("\n⚠️ Could not fetch work center data from the database. Final DataFrame is empty.")
    else:
        print("ℹ️ No rows found in T_A10 with POAURP = 2. Nothing to process.")

else:
    print("⚠️ T_A10 DataFrame is empty, skipping operation.")

print(f"The number of rows is: {len(A20_AprisoWorkCenter)}")

--- Joining T_A10 with MPDWCT to get Work Center details ---
✅ Connection successful to 'v12live' (v12Live).
Fetching details for 48 work centers from the database...


C:\Users\tdnguyen\AppData\Local\Temp\ipykernel_8956\1996041496.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_mpdwct = pd.read_sql_query(db_query, connection)


✅ Found 56 matching work centers.
🔌 Connection closed.

✅ Join successful. Final DataFrame created.


,PlannerCode,ppplgd,poopds,poaurp
0,MP-4320,4324R01,PRINTING TAMPON 1,2.0
1,MP-4320,4117R01,PREPARATION MATERIAL,2.0
2,MP-4320,4320R01,ASSEMBLY-THR,2.0
3,MP-4320,4320R01,FIRST SAMPLE INSPECTION,2.0
4,MP-4320,4323R01,TESTING HV,2.0


The number of rows is: 8502


### Step 17: Find Unique Apriso Work Center
This step creates a summary table named `A21_AprisoWorkCenter`. It distills the previous table down to only the unique combinations of planner codes, work center descriptions, and operation descriptions, and then sorts the result.

In [231]:
# Initialize the final DataFrame
A21_AprisoWorkCenter = pd.DataFrame()

if not A20_AprisoWorkCenter.empty:
    print("--- Finding unique combinations of PlannerCode, Work Center, and Operation ---")
    
    try:
        # Define the columns we are interested in
        # Note: The actual column names in our DataFrame are lowercase
        columns_to_check = ['PlannerCode', 'ppplgd', 'poopds']
        
        # 1. Select the columns, drop duplicate rows, and sort the result
        A21_AprisoWorkCenter = (
            A20_AprisoWorkCenter[columns_to_check]
            .drop_duplicates()
            .sort_values(by=columns_to_check)
            .reset_index(drop=True)
        )
        A21_AprisoWorkCenter = A21_AprisoWorkCenter.sort_values([ 'PlannerCode', 'ppplgd'])
        print("✅ Found unique combinations successfully.")
        display(A21_AprisoWorkCenter.head())

    except Exception as e:
        print(f"❌ An error occurred: {e}")

else:
    print("⚠️ A20_AprisoWorkCenter is empty, skipping operation.")

print(f"The number of rows is: {len(A21_AprisoWorkCenter)}")

--- Finding unique combinations of PlannerCode, Work Center, and Operation ---
✅ Found unique combinations successfully.


,PlannerCode,ppplgd,poopds
0,MP-4210,0000R01D,DISPATCH
1,MP-4210,3321R01,SOLDERING - SELECTIVE
2,MP-4210,4117R01,ASSEMBLY
3,MP-4210,4117R01,PREPARATION MATERIAL
4,MP-4210,4117R01,SELECTING MATERIAL


The number of rows is: 292


### Step 18: Extract '916' Merging Operations
This step creates a new DataFrame, `A20_Merging_XStep`, by filtering the main operations table `T_A10`. It isolates all rows where the operation is linked to a document ID (`PODOID`) that starts with "916", effectively creating a list of all "MERGING" steps for our analysis.

In [232]:
# Initialize the final DataFrame
A30_Merging_XStep = pd.DataFrame()

if not T_A10.empty:
    print("--- Filtering T_A10 for operations linked to '916' documents ---")
    
    try:
        # --- 1. WHERE clause ---
        # Filter rows where the 'podoid' column starts with "916"
        mask = T_A10['podoid'].astype(str).str.startswith('916', na=False)
        filtered_df = T_A10[mask]
        
        # --- 2. SELECT and Rename columns ---
        # The original T_A10 has lowercase columns; we'll select them and rename to match the query.
        # As before, 'mbresp' is the source for 'PlannerCode'.
        columns_to_rename = {
            'mbresp': 'PlannerCode',
            'pocono': 'POCONO',
            'pofaci': 'POFACI',
            'poprno': 'POPRNO',
            'poopno': 'POOPNO',
            'poplgr': 'POPLGR',
            'poopds': 'POOPDS',
            'podoid': 'PODOID'
        }
        
        A30_Merging_XStep = filtered_df.rename(columns=columns_to_rename)
        
        # Ensure we only keep the columns that were in the rename map
        final_columns = list(columns_to_rename.values())
        A30_Merging_XStep = A30_Merging_XStep[final_columns]
        
        print("✅ Filtering successful. A20_Merging_XStep DataFrame created.")
        display(A30_Merging_XStep.head())

    except Exception as e:
        print(f"❌ An error occurred: {e}")

else:
    print("⚠️ T_A10 DataFrame is empty, skipping operation.")

print(f"The number of rows is: {len(A30_Merging_XStep)}")

--- Filtering T_A10 for operations linked to '916' documents ---
✅ Filtering successful. A20_Merging_XStep DataFrame created.


,PlannerCode,POCONO,POFACI,POPRNO,POOPNO,POPLGR,POOPDS,PODOID
37,MP-4320,1.0,MF1,253994,2850.0,4320,MERGING,916-A2L0
120,MP-4320,1.0,MF1,096045,2830.0,4320,MERGING,916-A3H2
158,MP-4320,1.0,MF1,096046,2822.0,4320,MERGING,916-A3H2
201,MP-4320,1.0,MF1,038975,2835.0,4320,MERGING,916-A3H2
241,MP-4320,1.0,MF1,115621,2822.0,4320,MERGING,916-A3H2


The number of rows is: 483


### Step 19: Count Merging Steps per Part Number
This step creates the `A31_Merging_XStep` DataFrame. It aggregates the results from the previous step by grouping by `PlannerCode`, `POCONO`, `POFACI`, and `POPRNO`, and then counts the number of merging steps (`PODOID`) within each group.

In [233]:
# Initialize the final DataFrame
A31_Merging_XStep = pd.DataFrame()

if not A30_Merging_XStep.empty:
    print("--- Grouping and counting merging steps per part number ---")
    
    try:
        # Define the columns to group by
        grouping_cols = ['PlannerCode', 'POCONO', 'POFACI', 'POPRNO']
        
        # Group by the specified columns, count the 'PODOID' for each group,
        # and then reset the index to turn the grouped columns back into regular columns.
        A31_Merging_XStep = (
            A30_Merging_XStep.groupby(grouping_cols)['PODOID']
            .count()
            .reset_index()
            .rename(columns={'PODOID': 'Cnt_MergingStep'})
        )
        A31_Merging_XStep = A31_Merging_XStep.sort_values([ 'PlannerCode', 'POPRNO'])
        print("✅ Grouping and counting successful.")
        display(A31_Merging_XStep.head())
        print(f"The number of rows is: {len(A31_Merging_XStep)}")
        A31_Merging_XStep_More_Than_One = A31_Merging_XStep[A31_Merging_XStep['Cnt_MergingStep'] > 1]
        display(A31_Merging_XStep_More_Than_One.head())
        print(f"The number of rows is: {len(A31_Merging_XStep_More_Than_One)}")

    except Exception as e:
        print(f"❌ An error occurred: {e}")

else:
    print("⚠️ A20_Merging_XStep is empty, skipping operation.")


--- Grouping and counting merging steps per part number ---
✅ Grouping and counting successful.


,PlannerCode,POCONO,POFACI,POPRNO,Cnt_MergingStep
0,MP-4210,1.0,MF1,131000,2
1,MP-4210,1.0,MF1,131001,1
2,MP-4210,1.0,MF1,180563,3
3,MP-4210,1.0,MF1,189516,1
4,MP-4210,1.0,MF1,200354,2


The number of rows is: 392


,PlannerCode,POCONO,POFACI,POPRNO,Cnt_MergingStep
0,MP-4210,1.0,MF1,131000,2
2,MP-4210,1.0,MF1,180563,3
4,MP-4210,1.0,MF1,200354,2
24,MP-4210,1.0,MF1,239931,2
25,MP-4210,1.0,MF1,239935,2


The number of rows is: 76


### Step 20: Fetch Bill of Materials (BOM) for All Products
This step creates the `B10_BOM` DataFrame. It retrieves the component list (Bill of Materials) for every parent product in our initial `T_A00` list by querying the `MPDMAT` (BOM) and `MITMAS` (Item Master) tables from the database.

In [234]:
# Initialize the final DataFrame
B10_BOM = pd.DataFrame()

if not T_A00.empty:
    print("--- Fetching Bill of Materials data from the database ---")
    
    # --- 1. Get the list of parent products from T_A00 ---
    parent_products = T_A00['poprno'].unique().tolist()
    formatted_parents = ", ".join([f"'{p}'" for p in parent_products])
    
    # --- 2. Construct a targeted SQL query ---
    bom_sql_query = f"""
    SELECT
        T1.PMCONO, T1.PMFACI, T1.PMPRNO, T1.PMSTRT,
        T1.PMMSEQ, T1.PMFDAT, T1.PMTDAT, T1.PMMTNO, T1.PMCNQT,
        T2.MMITDS, T2.MMACRF, T2.MMINDI
    FROM
        MVXCDTA.MPDMAT AS T1
    INNER JOIN
        MVXCDTA.MITMAS AS T2 ON T1.PMMTNO = T2.MMITNO AND T1.PMCONO = T2.MMCONO
    WHERE
        T1.PMPRNO IN ({formatted_parents})
        AND T1.PMSTRT = 'STD'
        AND T1.PMTDAT = 99999999
    """
    
    df_bom_data = pd.DataFrame()
    connection = get_db_connection('v12live') # Re-establish connection
    if connection:
        try:
            print(f"Fetching BOM for {len(parent_products)} parent products...")
            df_bom_data = pd.read_sql_query(bom_sql_query, connection)
            df_bom_data.columns = df_bom_data.columns.str.lower()
            print(f"✅ Query successful! Found {len(df_bom_data)} BOM components.")
        except Exception as e:
            print(f"❌ Error executing database query: {e}")
        finally:
            connection.close()
            print("🔌 Connection closed.")
            
    # --- 3. Join database results with T_A00 to add PlannerCode and PLC ---
    if not df_bom_data.empty:
        # We only need a few columns from T_A00 for the join
        t_a00_subset = T_A00[['poprno', 'mbresp', 'plc']]
        
        merged_df = pd.merge(
            left=df_bom_data,
            right=t_a00_subset,
            left_on='pmprno',
            right_on='poprno',
            how='inner'
        )
        
        # --- 4. Final SELECT and Rename ---
        final_columns_map = {
            'mbresp': 'PlannerCode',
            'plc': 'PLC',
            'pmcono': 'PMCONO',
            'pmfaci': 'PMFACI',
            'pmprno': 'PMPRNO',
            'pmstrt': 'PMSTRT',
            'pmmseq': 'PMMSEQ',
            'pmfdat': 'PMFDAT',
            'pmtdat': 'PMTDAT',
            'pmmtno': 'PMMTNO',
            'mmitds': 'PMITDS',
            'pmcnqt': 'PMCNQT',
            'mmacrf': 'MMACRF',
            'mmindi': 'MMINDI'
        }
        
        B10_BOM = merged_df.rename(columns=final_columns_map)
        B10_BOM = B10_BOM.sort_values(['PlannerCode', 'PMPRNO', 'PMMSEQ'])
        
        # Ensure correct column order and selection
        B10_BOM = B10_BOM[list(final_columns_map.values())]
        
        print("\n✅ Final B10_BOM DataFrame created successfully.")
        display(B10_BOM.head())
    else:
        print("\n⚠️ No BOM data was fetched from the database.")
else:
    print("⚠️ T_A00 is empty, cannot fetch BOM data.")

print(f"The number of rows is: {len(B10_BOM)}")

--- Fetching Bill of Materials data from the database ---


✅ Connection successful to 'v12live' (v12Live).
Fetching BOM for 833 parent products...


C:\Users\tdnguyen\AppData\Local\Temp\ipykernel_8956\1576215992.py:32: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_bom_data = pd.read_sql_query(bom_sql_query, connection)


✅ Query successful! Found 23202 BOM components.
🔌 Connection closed.

✅ Final B10_BOM DataFrame created successfully.


,PlannerCode,PLC,PMCONO,PMFACI,PMPRNO,PMSTRT,PMMSEQ,PMFDAT,PMTDAT,PMMTNO,PMITDS,PMCNQT,MMACRF,MMINDI
3351,MP-4210,310,1.0,MF1,098916,STD,10.0,20170518.0,99999999.0,038188,C330P 400V 20%VDE D0800607P5,1.0,.0,0.0
3352,MP-4210,310,1.0,MF1,098916,STD,20.0,20170518.0,99999999.0,038188,C330P 400V 20%VDE D0800607P5,1.0,.0,0.0
4664,MP-4210,310,1.0,MF1,098916,STD,40.0,0.0,99999999.0,087668,FU 0600 L3P 2036-60-B 7505,1.0,.0,0.0
4661,MP-4210,310,1.0,MF1,098916,STD,50.0,0.0,99999999.0,087594,NSU62 BL 1.5KE62CA DO201,1.0,.0,0.0
4644,MP-4210,310,1.0,MF1,098916,STD,100.0,20220830.0,99999999.0,087399,PCB-RG-L02-A0012 F*-LB-I,1.0,.0,0.0


The number of rows is: 23202


### Step 21: Summarize Component Usage
This step creates the `B11_PMMTNO_group` DataFrame by aggregating the BOM data from the previous step. It groups by each unique component (`PMMTNO`) and its attributes, then calculates a UsageCnt to show how many different parent products use that specific component. The final result is sorted by the component's description.

In [235]:
# Initialize the final DataFrame
B11_PMMTNO_group = pd.DataFrame()

if not B10_BOM.empty:
    print("--- Grouping and counting component usage from BOM ---")
    
    try:
        # Define the columns to group by
        grouping_cols = [
            'PlannerCode', 
            'PMMTNO', 
            'PMITDS', 
            'MMACRF', 
            'MMINDI'
        ]
        
        # Group by the component details, count their usage, sort the result, and clean up the index
        B11_PMMTNO_group = (
            B10_BOM.groupby(grouping_cols)['PMCNQT']
            .count()
            .reset_index()
            .rename(columns={'PMCNQT': 'UsageCnt'})
            .sort_values(by='PMITDS')
            .reset_index(drop=True)
        )
        B11_PMMTNO_group = B11_PMMTNO_group.sort_values(['PlannerCode', 'PMMTNO'])
        print("✅ Component usage summary created successfully.")
        display(B11_PMMTNO_group.head())

    except Exception as e:
        print(f"❌ An error occurred: {e}")

else:
    print("⚠️ B10_BOM DataFrame is empty, skipping operation.")

print(f"The number of rows is: {len(B11_PMMTNO_group)}")

--- Grouping and counting component usage from BOM ---
✅ Component usage summary created successfully.


,PlannerCode,PMMTNO,PMITDS,MMACRF,MMINDI,UsageCnt
3250,MP-4210,002587,WIRE JUMPER 0.60X52.4,.0,0.0,3
624,MP-4210,002623,F40MA 250V T LC TR5 5P08,.0,0.0,1
2930,MP-4210,004447,SCREW YSMT03.0X025N/NC I 1207,.0,0.0,6
214,MP-4210,004573,CHM GLUE SICOMET 50 GG,.Z0,0.0,1
3227,MP-4210,004607,TUBE 1M00X0M25 CL SIL,.Z0,0.0,1


The number of rows is: 3283


### Step 22: Count Lot-Controlled Materials in BOM
This step creates the `B12_BOM_Lot_Ctr` DataFrame. Its purpose is to analyze the components from the previous summary and count how many of them are subject to lot control (where `MMINDI` is not 0). The result is a summary grouped by `PlannerCode`, `MMACRF` (the accounting control object), and the specific lot control method (`MMINDI`).

In [236]:
# Initialize the final DataFrame
B12_BOM_Lot_Ctr = pd.DataFrame()

if not B11_PMMTNO_group.empty:
    print("--- Counting lot-controlled components from the BOM summary ---")
    
    try:
        # --- 1. HAVING clause (applied as a WHERE filter first for efficiency) ---
        # Filter for rows where MMINDI is not 0
        filtered_df = B11_PMMTNO_group[B11_PMMTNO_group['MMINDI'] != 0].copy()

        # --- 2. GROUP BY and COUNT ---
        grouping_cols = ['PlannerCode', 'MMACRF', 'MMINDI']
        
        # --- 3. Chain all operations: Group, Count, Rename, and Sort ---
        B12_BOM_Lot_Ctr = (
            filtered_df.groupby(grouping_cols)['PMMTNO']
            .count()
            .reset_index()
            .rename(columns={'PMMTNO': 'CntLotCtrl'})
            .sort_values(by=['MMACRF', 'MMINDI'])
            .reset_index(drop=True)
        )
        B12_BOM_Lot_Ctr = B12_BOM_Lot_Ctr.sort_values(['PlannerCode', 'MMACRF'])
        print("✅ Summary of lot-controlled components created successfully.")
        display(B12_BOM_Lot_Ctr.head())

    except Exception as e:
        print(f"❌ An error occurred: {e}")

else:
    print("⚠️ B11_PMMTNO_group DataFrame is empty, skipping operation.")

print(f"The number of rows is: {len(B12_BOM_Lot_Ctr)}")

--- Counting lot-controlled components from the BOM summary ---
✅ Summary of lot-controlled components created successfully.


,PlannerCode,MMACRF,MMINDI,CntLotCtrl
0,MP-4210,.0,1.0,4
9,MP-4210,.2,3.0,10
12,MP-4210,.4B,3.0,3
19,MP-4210,.4M,3.0,1
23,MP-4210,.4S,1.0,3


The number of rows is: 46


### Step 23: Fetch Filtered Employee List
This step creates the E01_Employee DataFrame by querying the `MVXCDTA_CEAEMP` table from the database. The goal is to retrieve a specific list of employees who work in facility MF1 and belong to planner groups starting with "42", "43", or "45".

In [237]:
# Initialize the final DataFrame
E01_Employee = pd.DataFrame()

# This SQL query will be executed directly against the database
employee_sql_query = """
SELECT 
    EACONO, EADIVI, EAEMNO, EAEMNM, EACANO, EAFACI, 
    EAPLGR, EADEPT, EAREAR, EAACEM
FROM 
    MVXCDTA.CEAEMP
WHERE 
    EACONO = 1
    AND EAFACI = 'MF1'
    AND (EAPLGR LIKE '42%' OR EAPLGR LIKE '43%' OR EAPLGR LIKE '45%')
    AND EAACEM = 1
"""

connection = get_db_connection('v12live') # Assuming 'v12live' is the correct DSN
if connection:
    try:
        print("--- Fetching employee data from the database ---")
        E01_Employee = pd.read_sql_query(employee_sql_query, connection)
        
        # Standardize column names to lowercase for consistency in pandas
        E01_Employee.columns = E01_Employee.columns.str.lower()
        
        print(f"✅ Query successful! Found {len(E01_Employee)} matching employees.")
        display(E01_Employee.head())
        
    except Exception as e:
        print(f"❌ Error executing database query: {e}")
    finally:
        connection.close()
        print("🔌 Connection closed.")
else:
    print("⚠️ Could not connect to the database. Cannot fetch employee data.")

print(f"The number of rows is: {len(E01_Employee)}")

✅ Connection successful to 'v12live' (v12Live).
--- Fetching employee data from the database ---


C:\Users\tdnguyen\AppData\Local\Temp\ipykernel_8956\4208529220.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  E01_Employee = pd.read_sql_query(employee_sql_query, connection)


✅ Query successful! Found 172 matching employees.


,eacono,eadivi,eaemno,eaemnm,eacano,eafaci,eaplgr,eadept,earear,eaacem
0,1.0,100,MF10003693,Chan Kwai Lin,3693.0,MF1,4210,A-FIELDB,1-FBRI,1.0
1,1.0,100,MF10046995,Hu CaiYuan,46995.0,MF1,4210,A-FIELDB,1-FBRI,1.0
2,1.0,100,MF10047412,Lu Yu,47412.0,MF1,4210,A-FIELDB,1-FBRI,1.0
3,1.0,100,MF10047652,Lin ChunYan,47652.0,MF1,4210,A-FIELDB,2-FBRI,1.0
4,1.0,100,MF10074017,Toh Chuen Huey,74017.0,MF1,4210,A-FIELDB,2-FBRI,1.0


🔌 Connection closed.
The number of rows is: 172


### Step 24: Fetch Serial Number Request Data
This step creates the `S01_SerialNo_Request` DataFrame. It fetches all serial number requests made since the start of 2023 for the products in our `T_A00` list. This involves querying and joining two new tables, `PFXCDTA_SN5REQ` and `PFXCDTA_SN5RHD`, and then linking the results back to our initial product list.

In [238]:
# Initialize the final DataFrame
S01_SerialNo_Request = pd.DataFrame()

if not T_A00.empty:
    print("--- Fetching Serial Number Request data from the database ---")
    
    # 1. Get the list of products from T_A00 to use in our query
    products_in_scope = T_A00['poprno'].unique().tolist()
    formatted_products = ", ".join([f"'{p}'" for p in products_in_scope])

    # 2. Construct a targeted SQL query to get data from the two new tables
    sn_sql_query = f"""
    SELECT
        T1.SRRFUS, T1.SRRFID, T1.SRFACI, T1.SRWHLO, T1.SRRORC, T1.SRRONO,
        T1.SRITNO, T1.SRRGDT, T1.SRTPPF, T1.SRDOID, T1.SRTYP1, T1.SRTYP2,
        T1.SRTYP3, T1.SRTYP4, T1.SRNQTY, T1.SRREFT,
        T2.SAFRPF, T2.SATOPF, T2.SAFRT1, T2.SATOT1
    FROM
        PFXCDTA.SN5REQ AS T1
    INNER JOIN
        PFXCDTA.SN5RHD AS T2 ON T1.SRRFID = T2.SARFID
    WHERE
        T1.SRRGDT > 20230000
        AND T1.SRITNO IN ({formatted_products})
    """
    
    df_sn_data = pd.DataFrame()
    connection = get_db_connection('v12live') # Assuming 'v12live' DSN
    if connection:
        try:
            print(f"Fetching serial number requests for {len(products_in_scope)} products...")
            df_sn_data = pd.read_sql_query(sn_sql_query, connection)
            df_sn_data.columns = df_sn_data.columns.str.lower()
            print(f"✅ Query successful! Found {len(df_sn_data)} matching requests.")
        except Exception as e:
            print(f"❌ Error executing database query: {e}")
        finally:
            connection.close()
            print("🔌 Connection closed.")
            
    # 3. Join the new data with T_A00 to add PlannerCode and other details
    if not df_sn_data.empty:
        # We need PlannerCode (mbresp), Product Number (poprno), and Description (mmitds) from T_A00
        t_a00_subset = T_A00[['poprno', 'mbresp', 'mmitds']]
        
        merged_df = pd.merge(
            left=df_sn_data,
            right=t_a00_subset,
            left_on='sritno',
            right_on='poprno',
            how='inner'
        )
        
        # 4. Final SELECT and Rename to match the desired output
        final_columns_map = {
            'mbresp': 'PlannerCode', 'srrfus': 'SRRFUS', 'srrfid': 'SRRFID', 'srfaci': 'SRFACI',
            'srwhlo': 'SRWHLO', 'srrorc': 'SRRORC', 'srrono': 'SRRONO', 'sritno': 'SRITNO',
            'poprno': 'POPRNO', 'mmitds': 'MMITDS', 'srrgdt': 'SRRGDT', 'srtppf': 'SRTPPF',
            'srdoid': 'SRDOID', 'srtyp1': 'SRTYP1', 'srtyp2': 'SRTYP2', 'srtyp3': 'SRTYP3',
            'srtyp4': 'SRTYP4', 'srnqty': 'SRNQTY', 'srreft': 'SRREFT', 'safrpf': 'SAFRPF',
            'satopf': 'SATOPF', 'safrt1': 'CustomerSN', 'satot1': 'SATOT1'
        }
        
        S01_SerialNo_Request = merged_df.rename(columns=final_columns_map)
        
        # Ensure correct column order
        S01_SerialNo_Request = S01_SerialNo_Request[list(final_columns_map.values())]
        
        print("\n✅ Final S01_SerialNo_Request DataFrame created successfully.")
        display(S01_SerialNo_Request.head())
    else:
        print("\n⚠️ No serial number request data was fetched from the database.")
else:
    print("⚠️ T_A00 is empty, cannot fetch serial number data.")

print(f"The number of rows is: {len(S01_SerialNo_Request)}")

--- Fetching Serial Number Request data from the database ---
✅ Connection successful to 'v12live' (v12Live).
Fetching serial number requests for 833 products...


C:\Users\tdnguyen\AppData\Local\Temp\ipykernel_8956\2153816165.py:32: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sn_data = pd.read_sql_query(sn_sql_query, connection)


✅ Query successful! Found 10342 matching requests.
🔌 Connection closed.

✅ Final S01_SerialNo_Request DataFrame created successfully.


,PlannerCode,SRRFUS,SRRFID,SRFACI,SRWHLO,SRRORC,SRRONO,SRITNO,POPRNO,MMITDS,...,SRTYP1,SRTYP2,SRTYP3,SRTYP4,SRNQTY,SRREFT,SAFRPF,SATOPF,CustomerSN,SATOT1
0,MP-4210,PF_ETI 2.2.10,32BB747B-D637-49AA-8ACE-99AD97A1275D,MF1,MF1,1.0,7.002603e+09,204715-0010,204715-0010,RD0-FB-Ex4.COM,...,0.0,0.0,0.0,0.0,2.0,,40000155426527 ...,40000155426528 ...,...,...
1,MP-4210,PF_ETI 2.2.10,ECFA35B8-FE2C-47D9-B094-0D52D5ECB6E0,MF1,MF1,1.0,7.002603e+09,204715-0010,204715-0010,RD0-FB-Ex4.COM,...,0.0,0.0,0.0,0.0,5.0,,40000155426541 ...,40000155426545 ...,...,...
2,MP-4210,PF_ETI 2.2.10,CE6BEDB2-48FA-49D8-A36B-4045E264C14E,MF1,MF1,1.0,7.002603e+09,204715-0010,204715-0010,RD0-FB-Ex4.COM,...,0.0,0.0,0.0,0.0,3.0,,40000155426552 ...,40000155426554 ...,...,...
3,MP-4210,PF_ETI 2.2.10,92254280-8861-4060-8BEA-C0DE19181235,MF1,MF1,1.0,7.002603e+09,204715-0010,204715-0010,RD0-FB-Ex4.COM,...,0.0,0.0,0.0,0.0,5.0,,40000155426555 ...,40000155426559 ...,...,...
4,MP-4210,PF_ETI 2.2.10,25A0C7C3-F713-48BF-A26D-3A3A69686D96,MF1,MF1,1.0,7.002603e+09,204715-0010,204715-0010,RD0-FB-Ex4.COM,...,0.0,0.0,0.0,0.0,5.0,,40000155426566 ...,40000155426570 ...,...,...


The number of rows is: 10342


### Step 25: Count Product with print out serial number
This step creates the `S02_Product_Cnt_req` DataFrame. It aggregates the data from the previous step to count how many individual serial number requests (`SRRFUS`) were made for each unique combination of `PlannerCode`, `SRITNO` (Item Number), `MMITDS` (Description), and `SRDOID` (Document ID). From here we can identify how many item needs to print out serial number.

In [239]:
# Initialize the final DataFrame
S02_Product_Cnt_req = pd.DataFrame()

if not S01_SerialNo_Request.empty:
    print("--- Counting serial number requests per product and document ---")
    
    try:
        # Define the columns to group by
        grouping_cols = [
            'PlannerCode', 
            'SRITNO', 
            'MMITDS', 
            'SRDOID'
        ]
        
        # Group by the specified columns, count the requests, and clean up the DataFrame
        S02_Product_Cnt_req = (
            S01_SerialNo_Request.groupby(grouping_cols)['SRRFUS']
            .count()
            .reset_index()
            .rename(columns={'SRRFUS': 'AnzahlvonSRRFUS'})
        )
        S02_Product_Cnt_req = S02_Product_Cnt_req.sort_values(['PlannerCode', 'SRITNO'])
        print("✅ Request count summary created successfully.")
        display(S02_Product_Cnt_req.head())

    except Exception as e:
        print(f"❌ An error occurred: {e}")

else:
    print("⚠️ S01_SerialNo_Request DataFrame is empty, skipping operation.")

print(f"The number of rows is: {len(S02_Product_Cnt_req)}")

--- Counting serial number requests per product and document ---
✅ Request count summary created successfully.


,PlannerCode,SRITNO,MMITDS,SRDOID,AnzahlvonSRRFUS
0,MP-4210,204715-0010,RD0-FB-Ex4.COM,10-APD5E,45
1,MP-4210,204715-0011,RD0-FB-Ex4,10-APD5E,23
2,MP-4210,204715-0013,SPD0-FB-EX4,10-B1V3D,89
3,MP-4210,263565,FDH-1 Handheld,60-6970A,31
4,MP-4210,283822,HD2-GTR-4PA.PN,908-AGZ5,17


The number of rows is: 105


### Final Step: Export All Results to Excel
This last piece of code will gather every DataFrame we've created throughout this analysis and save them into one Excel file named `analysis_output.xlsx`. Each DataFrame will be placed on its own worksheet, with the sheet name corresponding to the DataFrame's name.

This creates a complete, organized report of your entire product analysis that you can easily share or review.

In [ ]:
# 1. Create a dictionary of all the DataFrames to export.
# The key is the sheet name, and the value is the DataFrame object.
dataframes_to_export = {
    # Initial Data
    "Product_List": T_A00,
    "Routing": T_A10,
    "Doc_Usage": T_A11,
    
    # Initial Analysis
    "Variant_Check": A01_Variant_Check,
    "ACO_Check": A02_ACO_Check,
    "LotControled": A03_LotControled,
    "PLCCheck": A03_PLCCheck,
    
    # EDM Document Analysis
    "Document": A12_DoKID_EDM,
    # "A13_Doc_Type": A13_Doc_type,
    # "A14_Doc_Type_Pivot": A14_Doc_type,
    # "A15_EDM_Doc_Ext_Type": A15_EDM_Doc_ext_type,
    "Labels": A16_EDM_Doc_CAB_Labels,
    "916-": A17_916_Files,
    # "A18_EDM_Doc_KeyValue": A18_EDM_Doc_KeyValue_File,
    
    # Work Center Analysis
    # "A20_AprisoWorkCenter": A20_AprisoWorkCenter,
    "AWC": A21_AprisoWorkCenter,
    
    # Merging Step Analysis
    # "A20_Merging_XStep": A30_Merging_XStep,
    "MergingStep": A31_Merging_XStep,
    "MergingStepMoreThanOne": A31_Merging_XStep_More_Than_One,

    # Bill of Materials (BOM) Analysis
    "BOM": B10_BOM,
    # "B11_BOM_Component_Usage": B11_PMMTNO_group,
    "Raw Material with Lot control": B12_BOM_Lot_Ctr,
    
    # Employee Data
    "Employee": E01_Employee,
    
    # Serial Number Analysis
    # "S01_SerialNo_Request": S01_SerialNo_Request,
    "ProductWithSerial": S02_Product_Cnt_req
}

# 2. Set the output filename
output_filename = 'analysis_output.xlsx'

# 3. Use ExcelWriter to save each DataFrame to a different sheet
try:
    with pd.ExcelWriter(output_filename, engine='openpyxl') as writer:
        print(f"--- Exporting all DataFrames to {output_filename} ---")
        
        for sheet_name, df in dataframes_to_export.items():
            # Only write sheets that have data
            if not df.empty:
                print(f"Writing sheet: {sheet_name} ({len(df)} rows)")
                df.to_excel(writer, sheet_name=sheet_name, index=False)
            else:
                print(f"Skipping empty sheet: {sheet_name}")
                
    print(f"\n✅ Export complete! Your file is saved as '{output_filename}'.")

except Exception as e:
    print(f"❌ An error occurred during export: {e}")
    print("Please make sure you have 'openpyxl' installed by running: pip install openpyxl")
    
# --- TIMER END ---
end_time = time.time()
total_time = end_time - start_time
print("\n--- Total Execution Time ---")
print(f"The entire notebook took {total_time:.2f} seconds to run.")
# -----------------

NameError: name 'A31_Merging_XStep_MoreThanOne' is not defined